## 1. Import Packages

In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from datetime import datetime
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet201
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Add, AveragePooling2D, BatchNormalization, Conv2D, Dense, Dropout, Flatten
from tensorflow.keras.layers import GlobalAveragePooling2D, Layer, MaxPool2D, ReLU, Resizing
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import L2
from tqdm import tqdm

dataset_dir = 'C:\\Users\\Jing Pengwei\\Desktop\\DSA5204 Project\\Dataset\\Facial Attractiveness\\SCUT-FBP5500_v2\\'
scut_dir = 'C:\\Users\\Jing Pengwei\\Desktop\\DSA5204 Project\\Dataset\\Facial Attractiveness\\SCUT-FBP\\'
mebeauty_dir = 'C:\\Users\\Jing Pengwei\\Desktop\\DSA5204 Project\\Dataset\\Facial Attractiveness\\MEBeauty\\'

In [2]:
print(tf.__version__)
print("Num GPUs Available", len(tf.config.experimental.list_physical_devices('GPU')))

2.10.0
Num GPUs Available 1


## 2. Dataset Cleaning

In [ ]:
# Remove Background from Images

mp_drawing = mp.solutions.drawing_utils
mp_selfie_segmentation = mp.solutions.selfie_segmentation

file_list = glob.glob(dataset_dir + 'Images\\*')

BG_COLOR = (255, 255, 255)
with mp_selfie_segmentation.SelfieSegmentation(model_selection = 0) as selfie_segmentation:
    for i in tqdm(range(len(file_list))):
        file_name = file_list[i].split('\\')[-1]
        image = cv2.imread(file_list[i])
        image_height, image_width, _ = image.shape

        results = selfie_segmentation.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        
        #fg_image = np.zeros(image.shape, dtype=np.uint8)
        #fg_image[:] = MASK_COLOR
        bg_image = np.zeros(image.shape, dtype=np.uint8)
        bg_image[:] = BG_COLOR

        output_image = np.where(condition, image, bg_image)
        cv2.imwrite(dataset_dir + 'Cleaned_Images\\' + file_name, output_image)

## 3. Dataset Preparation - MT-ResNet Full

In [ ]:
all_ratings = pd.read_csv(dataset_dir + 'train_test_files\\All_labels.txt', sep = ' ', header = None)
all_ratings.columns = ['img_path', 'rating']

img_arr = np.zeros([len(all_ratings), 224, 224, 3])
y_arr = np.zeros([len(all_ratings), 2])

for i in tqdm(range(len(all_ratings))):
    file_name = all_ratings.iloc[i, 0]
    
    race = file_name[0]
    gender = file_name[1]
    
    if gender == 'M':
        y_arr[i, 1] = 0
    elif gender == 'F':
        y_arr[i, 1] = 1
        
    y_arr[i, 0] = all_ratings.iloc[i, 1]
    
    img = tf.io.read_file(dataset_dir + 'Cleaned_Images\\' + file_name)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.keras.layers.Resizing(224, 224)(img)
    img /= 255
    img_arr[i] = img
    
np.save(dataset_dir + 'train_img_arr.npy', img_arr)
np.save(dataset_dir + 'train_y_arr.npy', y_arr)

In [3]:
train_img_arr = np.load(dataset_dir + 'train_img_arr.npy')
train_y_arr = np.load(dataset_dir + 'train_y_arr.npy')

## 4. MobileNetV2-MT

In [4]:
class Custom_Dropout(Layer):
    def __init__(self, rate, **kwargs):
        super(Custom_Dropout, self).__init__(**kwargs)
        self.rate = rate

        def call(self, inputs, training = None):
            if training:
                return tf.nn.dropout(inputs, rate = self.rate)
            return inputs

        
        
class MobileNetV2_MT_Extension(Model):
    def __init__(self):
        super(MobileNetV2_MT_Extension, self).__init__()
        
        self.mobilenetv2 = MobileNetV2(include_top = False)
        
        self.fc2_rating = Dense(1)
        self.fc2_gender = Dense(1, activation = 'sigmoid')
        
        self.global_avg_pool = GlobalAveragePooling2D()
        self.flatten = Flatten()
        self.dropout = Custom_Dropout(0.3)
        
        
        
    def call(self, inputs, training = None):
        x = self.global_avg_pool(self.mobilenetv2(inputs))
        x = self.flatten(x)
        
        x_rating = self.fc2_rating(x)
        x_gender = self.fc2_gender(x)
        
        return x_rating, x_gender

## 5. Training Configuration & Setup

### 5.1 Loss Function Definition - MT-ResNet

In [5]:
Loss_Functions = {'output_1': 'mse', 'output_2': 'binary_crossentropy'}
Loss_Weights = {'output_1': 2, 'output_2': 1}

### 5.2 Callbacks Setup

In [6]:
def custom_scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.01) 

    
    
required_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience = 15, restore_best_weights = True),
    tf.keras.callbacks.LearningRateScheduler(custom_scheduler),
    tf.keras.callbacks.ReduceLROnPlateau(factor = 0.5, patience = 5, verbose = 1, cooldown = 2, min_lr = 0.0000001),
    tf.keras.callbacks.TerminateOnNaN()
]

### 5.3 Training Parameters Definition - MT-ResNet

In [7]:
adam_optimizer = Adam(learning_rate = 0.005)
model = MobileNetV2_MT_Extension()
model.compile(optimizer = adam_optimizer, loss = Loss_Functions, loss_weights = Loss_Weights)

## 6. Training Phase - MT-ResNet

In [8]:
model.fit(train_img_arr, [train_y_arr[:, 0], train_y_arr[:, 1]], batch_size = 25, 
          epochs = 1000, verbose = 1, callbacks = required_callbacks, 
          validation_split = 0.2)

Epoch 1/1000
176/176 [==============================] - 27s 98ms/step - loss: 1.9628 - output_1_loss: 0.7772 - output_2_loss: 0.4083 - val_loss: 1324.8024 - val_output_1_loss: 659.2043 - val_output_2_loss: 6.3940 - lr: 0.0050
Epoch 2/1000
176/176 [==============================] - 16s 90ms/step - loss: 0.5129 - output_1_loss: 0.1808 - output_2_loss: 0.1512 - val_loss: 1645.6917 - val_output_1_loss: 817.3048 - val_output_2_loss: 11.0817 - lr: 0.0050
Epoch 3/1000
176/176 [==============================] - 16s 90ms/step - loss: 0.4238 - output_1_loss: 0.1581 - output_2_loss: 0.1076 - val_loss: 1060.5388 - val_output_1_loss: 523.1833 - val_output_2_loss: 14.1724 - lr: 0.0050
Epoch 4/1000
176/176 [==============================] - 16s 90ms/step - loss: 0.3603 - output_1_loss: 0.1461 - output_2_loss: 0.0681 - val_loss: 1556.6373 - val_output_1_loss: 767.0629 - val_output_2_loss: 22.5113 - lr: 0.0050
Epoch 5/1000
176/176 [==============================] - 16s 90ms/step - loss: 0.2691 - output

Epoch 36/1000
176/176 [==============================] - 16s 92ms/step - loss: 0.0178 - output_1_loss: 0.0088 - output_2_loss: 1.6103e-04 - val_loss: 0.2098 - val_output_1_loss: 0.0906 - val_output_2_loss: 0.0287 - lr: 9.1681e-04
Epoch 37/1000
176/176 [==============================] - 16s 92ms/step - loss: 0.0172 - output_1_loss: 0.0085 - output_2_loss: 1.5221e-04 - val_loss: 0.2187 - val_output_1_loss: 0.0962 - val_output_2_loss: 0.0262 - lr: 9.0769e-04
Epoch 38/1000
176/176 [==============================] - 16s 91ms/step - loss: 0.0176 - output_1_loss: 0.0085 - output_2_loss: 4.9167e-04 - val_loss: 0.2243 - val_output_1_loss: 0.0943 - val_output_2_loss: 0.0358 - lr: 8.9866e-04
Epoch 39/1000
176/176 [==============================] - 16s 93ms/step - loss: 0.0155 - output_1_loss: 0.0076 - output_2_loss: 2.9098e-04 - val_loss: 0.2367 - val_output_1_loss: 0.0965 - val_output_2_loss: 0.0436 - lr: 8.8971e-04
Epoch 40/1000
176/176 [==============================] - 16s 93ms/step - loss: 0

In [9]:
model.save('MobileNetV2_MT', save_format = 'tf')

INFO:tensorflow:Assets written to: MobileNetV2_MT\assets


INFO:tensorflow:Assets written to: MobileNetV2_MT\assets


## 7. SCUT-FBP Test Dataset Preparation and Evaluation

In [ ]:
mp_selfie_segmentation = mp.solutions.selfie_segmentation

rating_df = pd.read_excel(scut_dir + 'Rating_Collection\\Attractiveness label.xlsx', header = 0)
img_arr = np.zeros([len(rating_df), 224, 224, 3])
y_arr = np.zeros([len(rating_df), 2])

for i in tqdm(range(len(rating_df))):
    num = rating_df.iloc[i, 0]
    img = cv2.imread(scut_dir + 'Data_Collection\\SCUT-FBP-' + str(num) + '.jpg')
    
    with mp_selfie_segmentation.SelfieSegmentation(model_selection = 0) as selfie_segmentation:
        results = selfie_segmentation.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        condition = np.stack((results.segmentation_mask, ) * 3, axis = -1) > 0.1
        bg_image = np.zeros(img.shape, dtype = np.uint8)
        bg_image[:] = (255, 255, 255)
        cleaned_img = np.where(condition, img, bg_image)
    
    cleaned_img = tf.keras.layers.Resizing(224, 224)(cleaned_img)
    cleaned_img /= 255
    img_arr[i] = cleaned_img
    
    y_arr[i, 0] = rating_df.iloc[i, 1]
    y_arr[i, 1] = 1
    
np.save(scut_dir + 'img_arr.npy', img_arr)
np.save(scut_dir + 'y_arr.npy', y_arr)

In [10]:
img_arr = np.load(scut_dir + 'img_arr.npy')
y_arr = np.load(scut_dir + 'y_arr.npy')

In [11]:
model.evaluate(img_arr, [y_arr[:, 0], y_arr[:, 1]])

16/16 [==============================] - 1s 28ms/step - loss: 0.2966 - output_1_loss: 0.1438 - output_2_loss: 0.0090


[0.29658475518226624, 0.1437881737947464, 0.009008388966321945]

## 8. MEBeauty Test Dataset Preparation

In [ ]:
mp_selfie_segmentation = mp.solutions.selfie_segmentation

rating_df = pd.read_csv(mebeauty_dir + 'landmarks.csv', header = 0)
#img_arr = np.zeros([len(rating_df), 224, 224, 3])
#y_arr = np.zeros([len(rating_df), 2])

img_list = []
y_list = []

for i in tqdm(range(len(rating_df))):
    file_path = rating_df.iloc[i, 1]
    file_name = '\\'.join(file_path.split('/')[5:])
    gender = file_path.split('/')[5]
        
    try:
        img = cv2.imread(mebeauty_dir + 'cropped_images\\images_crop_align_mtcnn\\' + file_name)
        '''
        with mp_selfie_segmentation.SelfieSegmentation(model_selection = 0) as selfie_segmentation:
            results = selfie_segmentation.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            condition = np.stack((results.segmentation_mask, ) * 3, axis = -1) > 0.1
            bg_image = np.zeros(img.shape, dtype = np.uint8)
            bg_image[:] = (255, 255, 255)
            cleaned_img = np.where(condition, img, bg_image)
        '''

        img = tf.keras.layers.Resizing(224, 224)(img)
        img /= 255
        #img_arr[i] = img
        img = img[np.newaxis, :, :, :]
        img_list.append(img)
        
        if gender == 'female':
            #y_arr[i, 1] = 1
            y_list.append(np.array([rating_df.iloc[i, 2] / 2, 1]).reshape(1, 2))
        else:
            #y_arr[i, 1] = 0
            y_list.append(np.array([rating_df.iloc[i, 2] / 2, 0]).reshape(1, 2))
            
        #y_arr[i, 0] = rating_df.iloc[i, 2]
    except:
        continue
    
img_arr = np.concatenate(img_list, axis = 0)
y_arr = np.concatenate(y_list, axis = 0)
    
np.save(mebeauty_dir + 'img_arr.npy', img_arr)
np.save(mebeauty_dir + 'y_arr.npy', y_arr)

In [12]:
img_arr = np.load(mebeauty_dir + 'img_arr.npy')
y_arr = np.load(mebeauty_dir + 'y_arr.npy')

In [13]:
model.evaluate(img_arr, [y_arr[:, 0], y_arr[:, 1]])

73/73 [==============================] - 2s 28ms/step - loss: 1.7622 - output_1_loss: 0.4676 - output_2_loss: 0.8271


[1.7621889114379883, 0.46755269169807434, 0.8270841836929321]